add necessary library and functions

In [ ]:
import pyvista as pv
import os
from pathlib import Path
import matplotlib.pyplot as plt
import matplotlib.image as mpimg
from comtypes.client import CreateObject
import ansys.speos.core as core
import ansys.speos.workflow as workflow


def open_file(file):
    dpf_instance = None
    if file.endswith("xmp") or file.endswith("XMP"):
        dpf_instance = CreateObject("XMPViewer.Application")
        dpf_instance.OpenFile(file)
        res = dpf_instance.ExportXMPImage(file + ".png", 1)
        if res:
            img = mpimg.imread(file + ".png")
            plt.imshow(img)
            plt.axis("off")  # turns off axes
            plt.axis("tight")  # gets rid of white border
            plt.axis("image")  # square up the image instead of filling the "figure" space
            plt.show()
    elif file.endswith("hdr") or file.endswith("HDR"):
        dpf_instance = CreateObject("HDRIViewer.Application")
        dpf_instance.OpenFile(file)
        dpf_instance.Show(1)
    elif file.endswith("png") or file.endswith("PNG"):
        img = mpimg.imread(file)
        plt.imshow(img)
        plt.axis("off")  # turns off axes
        plt.axis("tight")  # gets rid of white border
        plt.axis("image")  # square up the image instead of filling the "figure" space
        plt.show()
    return dpf_instance


def load_glb(glb_path):
    data = pv.read(glb_path)
    faces_data = []
    vertices_data = []
    normals_data = []
    for data_block in data:
        faces_data.append(data_block[0][0].regular_faces.flatten().tolist())
        vertices_data.append(data_block[0][0].points.flatten().tolist())
        normals_data.append(data_block[0][0].point_normals.flatten().tolist())
    return faces_data, vertices_data, normals_data

initiate speos client

In [ ]:
speos = core.Speos(host="localhost", port=50051)
workflow.clean_all_dbs(speos.client)

load cad file

In [ ]:
tests_data_path = Path(Path.cwd().parent, "assets")
glb_file_path = os.path.join(tests_data_path, "ref_tri.glb")
facets, vertices, normals = load_glb(glb_file_path)

create face data

In [ ]:
facets_info = facets[0]
vertices_info = vertices[0]
normals_info = normals[0]
vertices_info = [1000 * item for item in vertices_info]
prism_face_list = []
prism_face = speos.client.faces().create(message=core.face.FaceFactory.new(name="face{}".format(0),
                                                                           description="source from stl",
                                                                           vertices=vertices_info,
                                                                           facets=facets_info,
                                                                           normals=normals_info))
prism_face_list.append(prism_face)

facets_info = facets[1]
vertices_info = vertices[1]
normals_info = normals[1]
vertices_info = [1000 * item for item in vertices_info]
print(vertices_info)
source_face_list = []
source_face = speos.client.faces().create(message=core.face.FaceFactory.new(name="face{}".format(0),
                                                                            description="source from stl",
                                                                            vertices=vertices_info,
                                                                            facets=facets_info,
                                                                            normals=normals_info))
source_face_list.append(source_face)

create body data

In [ ]:
bd_prism = speos.client.bodies().create(message=core.body.BodyFactory.new(name="prism",
                                                                          description="triangle",
                                                                          faces=prism_face_list))
bd_source = speos.client.bodies().create(message=core.body.BodyFactory.new(name="source",
                                                                           description="triangle",
                                                                           faces=source_face_list))

create part data

In [ ]:
part1 = speos.client.parts().create(
    message=core.part.PartFactory.new(
        name="part_0",
        description="part with one box as body",
        bodies=[bd_prism, bd_source]))


create sop data

In [ ]:
sop_instances = [
    core.scene.SceneFactory.sop_instance(
        name="OP",
        sop_template=speos.client.sop_templates().create(
            message=core.sop_template.SOPTemplateFactory.optical_polished(
                name="OP",
                description="Opticalpolished")),
        geometries=core.geometry_utils.GeoPaths(geo_paths=[
            "prism/face0",
            "source/face0",
        ])
    )]

create vop data

In [ ]:
vop_instances = [
    core.scene.SceneFactory.vop_instance(
        name="Optic",
        vop_template=speos.client.vop_templates().create(
            message=core.vop_template.VOPTemplateFactory.optic(
                name="refractive",
                index=1.51,
                absorption=0.001,
                constringence=60.0)),
        geometries=core.geometry_utils.GeoPaths(geo_paths=["prism", "source"]),
    )
]

create source data

In [ ]:
spec_bb_5800 = speos.client.spectrums().create(
    message=core.spectrum.SpectrumFactory.blackbody(
        name="blackbody_5800",
        description="blackbody spectrum - T 3500K",
        temperature=5800))

# Create lambertian intensity template
intens_t_lamb_180 = speos.client.intensity_templates().create(
    message=core.intensity_template.IntensityTemplateFactory.lambertian(
        name="lambertian_180",
        description="lambertian intensity template 180",
        total_angle=0.1))

src_t_surface_bb = speos.client.source_templates().create(
    message=core.source_template.SourceTemplateFactory.surface(
        name="surface_with_monochromatic",
        description="Surface source template with blackbody spectrum",
        intensity_template=intens_t_lamb_180,
        flux=core.source_template.SourceTemplateFactory.Flux(),
        spectrum=spec_bb_5800,
    )
)

source_instance = [
    core.scene.SceneFactory.source_instance(
        name="whitesource_1",
        source_template=src_t_surface_bb,
        properties=core.scene.SceneFactory.surface_source_props(
            exitance_constant_geo_paths=[
                core.geometry_utils.GeoPathWithReverseNormal("source/face0")]),
    )
]

create sensor data

In [ ]:
ssr_t_irr_colo = speos.client.sensor_templates().create(
    message=core.sensor_template.SensorTemplateFactory.irradiance(
        name="irradiance_colorimetric",
        description="Irradiance sensor template spectral",
        type=core.sensor_template.SensorTemplateFactory.Type.Spectral,
        illuminance_type=core.sensor_template.SensorTemplateFactory.IlluminanceType.Planar,
        dimensions=core.sensor_template.SensorTemplateFactory.Dimensions(
            x_start=-10, x_end=10, x_sampling=200, y_start=-10, y_end=10, y_sampling=200
        ),
        wavelengths_range=core.sensor_template.SensorTemplateFactory.WavelengthsRange(start=380, end=780, sampling=41),
    )
)

irr_sensor_props = core.scene.SceneFactory.irradiance_sensor_props(
    axis_system=core.geometry_utils.AxisSystem(origin=[143.39, -500, 0.0],
                                               x_vect=[-1.0, 0.0, 0.0],
                                               y_vect=[0.0, 0.0, 1.0],
                                               z_vect=[0.0, -1.0, 0.0]),
    layer_type=core.scene.SceneFactory.Properties.Sensor.LayerType.Source(),
)

sensor_instance = [
    core.scene.SceneFactory.sensor_instance(
        name="irradiance_spectral",
        sensor_template=ssr_t_irr_colo,
        properties=irr_sensor_props)
]

create simulation data

In [ ]:
direct_t = speos.client.simulation_templates().create(
    message=core.simulation_template.SimulationTemplateFactory.direct_mc(name="direct_simu",
                                                                         description="Direct simulation template with "
                                                                                     "default parameters")
)

scene = speos.client.scenes().create(
    message=core.scene.SceneFactory.new(
        name="scene_0",
        description="scene from scratch",
        part=part1,
        vop_instances=vop_instances,
        sop_instances=sop_instances,
        source_instances=source_instance,
        sensor_instances=sensor_instance,
        simulation_instances=[
            core.scene.SceneFactory.simulation_instance(
                name="direct_simu.2",
                simulation_template=direct_t,
                source_paths=["whitesource_1"],
                sensor_paths=["irradiance_spectral"],
                geometries=core.geometry_utils.GeoPaths(["prism", "source"]))
        ]
    )
)


create simulation job to run

In [ ]:
props = core.JobFactory.direct_mc_props(stop_condition_duration=500)
new_job = speos.client.jobs().create(
    message=core.JobFactory.new(
        name="test",
        scene=scene,
        simulation_path="direct_simu.2",
        properties=props,
    )
)

new_job.start()
print(new_job)

wait for simulation to complete, get the xmp result

In [ ]:
print(new_job.get_results().results[0].path)

open and review the rainbow result

In [ ]:
open_file(new_job.get_results().results[0].path)